In [19]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
os.chdir('/Users/andreapiemontese/Desktop/Git/energy-economics')

In [20]:
df_hist_features = pd.read_csv("data/History/History_Features.csv", sep = ';')
df_hist_target = pd.read_csv("data/History/History_Targets.csv", sep=';')
print(df_hist_features)
print(df_hist_target)

    Year  Power_DE_Prices  Gas_TTF_Prices  CO2_EUA_Prices  Coal_API2_Prices
0   2014            32.76           20.92            5.94             74.96
1   2015            31.63           19.80            7.63             55.88
2   2016            28.98           14.01            5.29             59.20
3   2017            34.19           17.30            5.75             83.07
4   2018            44.47           22.80           15.72             91.64
5   2019            37.67           13.55           24.70             62.27
6   2020            30.47            9.40           24.62             51.43
7   2021            96.85           46.36           52.85            116.08
8   2022           235.45          122.88           79.89            277.45
9   2023            95.18           40.74           83.29            124.60
10  2024            79.62           34.35           64.94            111.79
    Year  Power_Demand_DE_Industry  Industrial_Production_Index_DE
0   2014             

In [24]:
df_hist_target.describe().merge(df_hist_features.describe().drop(columns=['Year']), left_index=True, right_index=True)

,Year,Power_Demand_DE_Industry,Industrial_Production_Index_DE,Power_DE_Prices,Gas_TTF_Prices,CO2_EUA_Prices,Coal_API2_Prices
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000
mean,2019.000000,226.090909,100.152727,67.933636,32.919091,33.692727,100.760909
std,3.316625,15.352228,4.122783,61.528802,32.070455,30.758192,63.923542
min,2014.000000,201.000000,92.710000,28.980000,9.400000,5.290000,51.430000
25%,2016.500000,214.500000,97.820000,32.195000,15.655000,6.785000,60.735000
50%,2019.000000,229.000000,99.730000,37.670000,20.920000,24.620000,83.070000
75%,2021.500000,239.500000,103.010000,87.400000,37.545000,58.895000,113.935000
max,2024.000000,242.000000,106.310000,235.450000,122.880000,83.290000,277.450000


In [25]:
fig = px.line(df_hist_target, x='Year', y='Power_Demand_DE_Industry',
              title='Industry Power Demand (TWh)')
fig.show()

In [26]:
fig = px.line(df_hist_target, x='Year', y='Industrial_Production_Index_DE',
              title='Industrial Production Index (%, indexed)')
fig.show()

In [ ]:
# Industrial Production Index
fig = px.line(df_hist_features, x='Year', y='Industrial_Production_Index_DE', title='Industrial Production Index (%, indexed)')
fig.show()


In [29]:
price_cols = ['Power_DE_Prices','Gas_TTF_Prices','CO2_EUA_Prices','Coal_API2_Prices']

for col in price_cols:
    fig = px.line(df_hist_features, x='Year', y=col, title=f'{col} over time')
    fig.show()

In [35]:
import plotly.figure_factory as ff

# Define columns to include in correlation
corr_cols = ['Power_Demand_DE_Industry', 'Industrial_Production_Index_DE',
             'Power_DE_Prices', 'Gas_TTF_Prices', 'CO2_EUA_Prices', 'Coal_API2_Prices']
df_hist = df_hist_target.merge(df_hist_features, how = 'inner', on = ['Year'])
# Compute correlation matrix
corr = df_hist[corr_cols].corr()
fig = ff.create_annotated_heatmap(
    z=corr.values,
    x=corr.columns.tolist(),
    y=corr.columns.tolist(),
    colorscale='Viridis',
    showscale=True,
    annotation_text=np.round(corr.values, 2)  # show rounded numbers
)

fig.update_layout(
    title='Correlation Heatmap: Historical Data',
    width=700,
    height=700
)

fig.show()


In [36]:
df_hist['Power_Demand_YoY'] = df_hist['Power_Demand_DE_Industry'].pct_change() * 100
df_hist['IPI_YoY'] = df_hist['Industrial_Production_Index_DE'].pct_change() * 100

# Plot
import plotly.express as px

fig = px.line(df_hist, x='Year', y='Power_Demand_YoY', title='Power Demand YoY Change (%)')
fig.show()

fig = px.line(df_hist, x='Year', y='IPI_YoY', title='Industrial Production Index YoY Change (%)')
fig.show()

In [38]:
df_hist['Period'] = np.where(df_hist['Year'] < 2022, 'Pre-2022', '2022+')
summary_by_period = df_hist.groupby('Period').agg({
    'Power_Demand_DE_Industry': ['mean','std'],
    'Industrial_Production_Index_DE': ['mean','std'],
    'Power_DE_Prices': ['mean','std']
})
summary_by_period

Power_Demand_DE_Industry           Industrial_Production_Index_DE  \
                             mean       std                           mean   
Period                                                                       
2022+                  205.666667  5.686241                      96.116667   
Pre-2022               233.750000  9.035644                     101.666250   

                   Power_DE_Prices             
               std            mean        std  
Period                                         
2022+     3.078706        136.7500  85.830041  
Pre-2022  3.460413         42.1275  22.645132